In [110]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climateedsatrain/train.csv
/kaggle/input/climateedsatest/test.csv
/kaggle/input/climate-edsa/test.csv
/kaggle/input/climate-change-edsa2020-21/train.csv
/kaggle/input/climate-change-edsa2020-21/test.csv
/kaggle/input/climate-change-edsa2020-21/sample_submission.csv


In [111]:
#import libs
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
# Classification report
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

In [112]:
#load files for train and test
train_df1 = pd.read_csv('/kaggle/input/climateedsatrain/train.csv')
test_df1 = pd.read_csv('/kaggle/input/climateedsatest/test.csv')

In [113]:
#check if file loaded
#print(train_df1)
#print(test_df1)

In [114]:
#make a copy
df1 = train_df1.copy()
#print(df1)
df2 = train_df1.copy()

df1.loc[df1['sentiment'] == -1, 'sentiment_word'] = 'Anti'
df1.loc[df1['sentiment'] == 0, 'sentiment_word'] = 'Neutral'
df1.loc[df1['sentiment'] == 1, 'sentiment_word'] = 'Pro'
df1.loc[df1['sentiment'] == 2, 'sentiment_word'] = 'News'
#print(df1.head())
#get hashtags from messages
#df1['hashtags'] = df1['message'].str.findall(r'#(\w+)')
#def findhashtag(messages):
news_check = df1['sentiment_word'] == 'News'
news = df1[news_check]
pro_check = df1['sentiment_word'] == 'Pro'
pro = df1[pro_check]
neutral_check = df1['sentiment_word'] == 'Neutral'
neutral = df1[neutral_check]
anti_check = df1['sentiment_word'] == 'Anti'
anti = df1[anti_check]
#print(news.head())
#print(pro.head())
#print(neutral.head())
#print(anti.head())
pro_tag = []
for i in pro['message']:
    hashes = re.findall(r"#(\w+)", i)
    pro_tag.append(hashes)
pro_tag = sum(pro_tag,[])
pro_freq = nltk.FreqDist(pro_tag)
pro_df = pd.DataFrame({'hashtags': list(pro_freq.keys()), 'counts': list(pro_freq.values())})
pro_df = pro_df.nlargest(15, columns="counts")
#pro_df = pd.DataFrame({'hashtags': pro_tag, 'counts': list(pro_freq.values())})
#News_tags = pd.DataFrame({'hashtags': list(freq.keys()), 'counts': list(freq.values())})
#print(pro_df)

news_tag = []
for i in news['message']:
    hashes = re.findall(r"#(\w+)", i)
    news_tag.append(hashes)
news_tag = sum(news_tag,[])
news_freq = nltk.FreqDist(news_tag)
news_df = pd.DataFrame({'hashtags': list(news_freq.keys()), 'counts': list(news_freq.values())})
news_df = news_df.nlargest(15, columns="counts")

anti_tag = []
for i in anti['message']:
    hashes = re.findall(r"#(\w+)", i)
    anti_tag.append(hashes)
anti_tag = sum(anti_tag,[])
anti_freq = nltk.FreqDist(anti_tag)
anti_df = pd.DataFrame({'hashtags': list(anti_freq.keys()), 'counts': list(anti_freq.values())})
anti_df = anti_df.nlargest(15, columns="counts")

neutral_tag = []
for i in neutral['message']:
    hashes = re.findall(r"#(\w+)", i)
    neutral_tag.append(hashes)
neutral_tag = sum(neutral_tag,[])
neutral_freq = nltk.FreqDist(neutral_tag)
neutral_df = pd.DataFrame({'hashtags': list(neutral_freq.keys()), 'counts': list(neutral_freq.values())})
neutral_df = neutral_df.nlargest(15, columns="counts")

print(news_df.head())
print(pro_df.head())
print(neutral_df.head())
print(anti_df.head())

         hashtags  counts
0         climate     130
6     environment      44
11  climatechange      42
57          Trump      25
47           news      20
           hashtags  counts
19          climate     187
24   BeforeTheFlood     129
68    climatechange      94
13  ImVotingBecause      62
4             COP22      59
          hashtags  counts
27         climate      16
9    climatechange      11
104          Trump      11
4    ClimateChange       4
70       amreading       4
         hashtags  counts
30           MAGA      11
48        climate      10
12          Trump       7
72  climatechange       6
7    OpChemtrails       4


In [115]:
#make the messages to lower case
df1['message'] = df1['message'].str.lower()

#df2['message'] = df2['message'].str.lower()
#df2['message'] = df2['message'].apply(lambda x: re.sub(r'@[\w]*', x))
#print(df2)
#
#for i in df1['sentiment']:
#    if i == 2:
#        df1['sentiments']='News'
#    elif i == 1:
#        df1['sentiments']='Pro'
#    elif i == 0:
#        df1['sentiments']='Neutral'
#    elif i == -1:
#        df1['sentiments']='Anti'

#print(df1.head(20))

#lemma words in message


#extract hastag using re
#add hashtag column
#df1['hashtags'] = df1['message'].str.findall(r'#.*?(?=\s|$)')
#df1['hashtags'] = df1['message'].str.findall(r'#(\w+)')
#df2 = df1.dropna()
#print(type(df1['hashtags']))
#print(df1)

#df1['hashtags'] = df1['hashtags'].to_string()
#df1['lowerHashtags'] = df1['hashtags'].str.lower()
#print(type(df1['message']))
#print(df1)
#df1
#df1['firstHashtags'] = df1['hashtags'].iloc[0]
#df1.astype(str)
#df1.dtypes
###################
#hashtags_only = []
#tweetMsg = df1['message']
#for i in tweetMsg:
#    hashtag = re.findall(r'#(\w+)', i)
#    hashtags_only.append(hashtag)
#hasht = sum(hashtags_only,[])
#freq = nltk.FreqDist(hasht)
#df1_hashtags = pd.DataFrame({'hashtag': list(freq.keys()), 'counts': list(freq.values())})
#print(df1_hashtags)
##########################
#hashtag only dataframe
#print(df1)
#hashtag_df1['hashtags'] = df1.explode('hashtags')
#hashtag_df1
#print(hashtag_df1)
#hashOnly['hashtags'] = hashtag_df1['hashtags']
#print(hashonly)
#hashOnly = hashOnly.dropna()
#print(hashonly)
#hashOnlyUnq = hashOnly.sort_values()
#print(hashOnlyUnq)
#hashDup = hashOnlyUnq.pivot_table(index=['hashtags'], addfunc='size')
#
#print(df1)

def preprocess(message):
    message = re.sub(r"#\w*", '', message)
    message = re.sub(r"\d+", '', message)
    message = re.sub(r"@[A-Za-z0-9]", '', message)
    message = re.sub(r"rt[\s]+", ' ', message)
    message = re.sub(r"https?:\/\/\S+", ' ', message)
    return message
df1['message'] = df1['message'].apply(preprocess)
print(df1.head())

   sentiment                                            message  tweetid  \
0          1  polyscimajor epa chief doesn't think carbon di...   625221   
1          1  it's not like we lack evidence of anthropogeni...   126103   
2          2   awstory: researchers say we have three years ...   698562   
3          1   wired :  was a pivotal year in the war on cli...   573736   
4          1   oynoviodetodas: it's , and a racist, sexist, ...   466954   

  sentiment_word  
0            Pro  
1            Pro  
2           News  
3            Pro  
4            Pro  


In [116]:
#tokenize, lemmanize
df1.head()
word_tokens = []
#for i in df1['message']:
#    word_tokens.append(word_tokenize(i))
df1['tokenized'] = df1['message'].apply(word_tokenize)
df1['len'] = df1['message'].str.len()
df1['speechTags'] = df1['tokenized'].apply(nltk.tag.pos_tag)
#df1['lemmanize'] = df1['message'].apply(WordNetLemmatizer)
w_token = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()
def lemmanizetext(text):
    return [lemmatizer.lemmatize(x) for x in w_token.tokenize(text)]
df1['text_lemma'] = df1.message.apply(lemmanizetext)
df1.head()

,sentiment,message,tweetid,sentiment_word,tokenized,len,speechTags,text_lemma
0,1,polyscimajor epa chief doesn't think carbon di...,625221,Pro,"[polyscimajor, epa, chief, does, n't, think, c...",116,"[(polyscimajor, JJ), (epa, NN), (chief, NN), (...","[polyscimajor, epa, chief, doesn't, think, car..."
1,1,it's not like we lack evidence of anthropogeni...,126103,Pro,"[it, 's, not, like, we, lack, evidence, of, an...",62,"[(it, PRP), ('s, VBZ), (not, RB), (like, IN), ...","[it's, not, like, we, lack, evidence, of, anth..."
2,2,awstory: researchers say we have three years ...,698562,News,"[awstory, :, researchers, say, we, have, three...",95,"[(awstory, NN), (:, :), (researchers, NNS), (s...","[awstory:, researcher, say, we, have, three, y..."
3,1,wired : was a pivotal year in the war on cli...,573736,Pro,"[wired, :, was, a, pivotal, year, in, the, war...",59,"[(wired, VBN), (:, :), (was, VBD), (a, DT), (p...","[wired, :, wa, a, pivotal, year, in, the, war,..."
4,1,"oynoviodetodas: it's , and a racist, sexist, ...",466954,Pro,"[oynoviodetodas, :, it, 's, ,, and, a, racist,...",100,"[(oynoviodetodas, NNS), (:, :), (it, PRP), ('s...","[oynoviodetodas:, it's, ,, and, a, racist,, se..."


In [117]:
X = train_df1['message']
y = train_df1['sentiment']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

In [118]:
y_valid

9059     1
169      2
5331     1
6333    -1
10383    0
        ..
1950     0
14723    1
651      2
5015     2
7567     2
Name: sentiment, Length: 3955, dtype: int64

In [119]:
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe = Pipeline
logreg = pipe([('vect',TfidfVectorizer()),
               ('logreg',LogisticRegression(C=1, max_iter=10000))])

In [120]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_valid)
y_test = logreg.predict(test_df1['message'])

In [121]:
submission = pd.DataFrame({'tweetid': test_df1['tweetid'], 'sentiment': y_test})
#submission.to_csv('/kaggle/input/climateedsatrain/submission.csv', index = False)
submission

,tweetid,sentiment
0,Europe will now be looking to China to make su...,1
1,Combine this with the polling of staffers re c...,1
2,"The scary, unimpeachable evidence that climate...",1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,0
...,...,...
10541,"RT @BrittanyBohrer: Brb, writing a poem about ...",1
10542,2016: the year climate change came home: Durin...,1
10543,RT @loop_vanuatu: Pacific countries positive a...,1
10544,"RT @xanria_00018: You’re so hot, you must be t...",0
